In [84]:
import csv
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None 

from funcs import driver_status
# from funcs import get_col_name
# from funcs import maxer
# from funcs import maxer_2
# from funcs import check_truck

In [2]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is B0D1-7E59

 Directory of C:\Users\hp\Desktop\due_dilligence_2

11/27/2022  04:14 PM    <DIR>          .
11/27/2022  04:14 PM    <DIR>          ..
11/27/2022  01:08 PM    <DIR>          .ipynb_checkpoints
11/27/2022  03:14 PM         7,763,232 data21.xlsx
11/27/2022  03:20 PM        14,439,045 data22.xlsx
11/27/2022  03:21 PM         1,524,300 datasep.xlsx
11/27/2022  04:14 PM            14,463 due_dilligence2_(driver_analysis).ipynb
11/27/2022  04:14 PM             1,481 funcs.py
11/27/2022  01:06 PM               734 Read Me.txt
11/27/2022  04:15 PM    <DIR>          __pycache__
               6 File(s)     23,743,255 bytes
               4 Dir(s)  19,714,174,976 bytes free


I had to break down the data from the question raw data in illa overview to 3 questions using illa overview's filters for 2021,2022 and sep2022 to download the 3 excels and use then merge them here to have all the transactional data in one df 

In [3]:
df1 = pd.read_excel("data21.xlsx")
df2 = pd.read_excel("data22.xlsx")
df3 = pd.read_excel("datasep.xlsx")

In [48]:
df_dstatus = pd.read_excel("driver_status.xlsx")
df_dstatus = df_dstatus.rename(columns = {"Driver Name":"driver_name"})

In [23]:
df["trips_status"].unique()

array(['Completed', 'Unfulfilled', 'Not_loaded', 'Not_requested',
       'Absent', 'Cancelled By C', 'Out for Delivery'], dtype=object)

In [89]:
df = pd.concat([df1,df2], axis = 0)
df = pd.concat([df,df3], axis = 0)
df["trip_date"]= pd.to_datetime(df["trip_date"])
df["month_year"] = df["trip_date"].dt.strftime('%m/%Y').astype('datetime64')
df= df[df["trips_status"].isin(["Completed", "Not_loaded","Not_requested"])]
df["truck"] = df["truck_type"].apply(check_truck)

In [26]:
print(df["trip_date"].min())
print(df["trip_date"].max())

2021-06-01 00:00:00
2022-09-30 00:00:00


In [44]:
df_combo = df.merge(df_dstatus[["driver_name","Driver Class"]], on ="driver_name")

In [86]:
### Needed functions
def get_col_name(row):    
    b = (df2.index[row.name] == row['value'])
    return b.index[b.argmax()]

def maxer(df,s=0,e=12):
    max_index = []
    for i in df.index.values:
        max_index.append(df.iloc[:,s:e].loc[str(i)].idxmax())
    return pd.Series(max_index, name = "max_month", index=df.index)

def maxer_2(df,s=0,e=12):
    max_index = []
    for i in df.index.values:
        max_index.append(df.iloc[:,s:e].iloc[i].idxmax())
    return pd.Series(max_index, name = "max_month", index=df.index)

def check_truck(val):
          val =  val.lower().replace('_'," ").replace("-"," ").split()
          if "jumbo" in val:
            return "jumbo"
          elif "dababa" in val:
            return "dababa"
          elif ("trella" in val)|("side" in val):
            return "trailer"
          elif "van" in val:
            return "van"

##### Slide2 __(active vs new drivers) (5 in pres)

In [28]:
########### We aggregate data according to month year  
driver_month_pivot =  df.pivot_table(index=["driver_name"], 
                                    columns=["month_year"], 
                                    values=["id"], 
                                    aggfunc="count", 
                                    fill_value=0)
driver_month_pivot = driver_month_pivot.applymap(lambda x: 1 if x>0 else 0)
########## Func driver status is imported from the module funcs in the same direcotry with two arguments
########## df and number of months(cols), it just shows the status of each driver everymonth 


driver_month_pivot= driver_month_pivot.reset_index()
cols  = [str(i[1])[:-9] for i in list(driver_month_pivot.columns)]
cols.insert(1,"driver")
cols= cols[1:]
driver_month_pivot.columns = cols
driver_month_pivot= driver_month_pivot.set_index("driver")
driver_month_status = pd.DataFrame(driver_month_pivot.apply(lambda x: pd.Series(driver_status(x,len(driver_month_pivot.columns))), axis=1))

driver_month_status.columns = driver_month_pivot.columns

driver_month_status
month_status_pivot = pd.DataFrame(driver_month_status.replace("unreg", np.NaN).apply(lambda x: pd.value_counts(x)))
month_status_pivot = month_status_pivot.fillna(0)
month_status_pivot 

,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01
active,0.0,123,170,181,213,221,215,261,283,285,370,334,338,347,342,293
inactive,0.0,134,285,438,561,686,909,1118,1290,1443,1707,2010,2230,2505,2736,2940
new,257.0,198,164,155,133,217,255,194,155,349,267,224,284,226,155,197


##### Slide3 __(driver count vs trips count) (6 in pres)

In [34]:
slide_1 = df.groupby(['month_year','driver_name']).count().groupby(['month_year']).count().reset_index().iloc[:,[0,1]].rename(columns={"id":"driver_count"}).merge(
df.groupby(['month_year','driver_name']).count().groupby(["month_year"]).sum().reset_index().iloc[:,[0,1]].rename(columns={"id":"trips_count"}), on = "month_year",how ="left").transpose()

# slide_1.columns = ["jun21", "jul21","aug21","sep21","oct21","nov21","dec21","jan22","feb22","mar22","apr22"]
###################### cols from slide 1 
slide_1.columns = cols[1:]
slide_1 = slide_1.drop(slide_1.index[[0]],axis= 0)
slide_1

,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01
driver_count,257,321,334,336,346,438,470,455,438,634,637,558,622,573,497,490
trips_count,2070,3188,4178,5218,4999,4575,4749,5083,5249,6482,7302,8071,6839,6829,7460,5722


##### Slide4__(first to last growth (7 in pres)

In [95]:
slide_1 = df.copy()

slide_1 = slide_1.groupby(['month_year']).agg({'driver_name':'nunique','id':'count', "manual_distance":"sum","client_name":"nunique","supplier_name":"nunique","branch_name":"nunique","pickup_gov":"nunique","truck_type":"nunique"})
slide_1.columns  =['driver_ucounts', 'trips_count',"distance_covered","clients_ucount","supp_ucount","branch_ucount", "gov_ucount","truck_types"]
slide_1 = slide_1.transpose()#.reset_index().iloc[:,[0,1]].rename(columns={"id":"driver_count"}).merge(slide_1.groupby(['month_year','driver_name']).count().groupby(["month_year"]).sum().reset_index().iloc[:,[0,1]].rename(columns={"id":"trips_count"}), on = "month_year",how ="left").transpose()
# slide_1.columns = cols[1:]
slide_1.columns = ["jun21","jul21","aug21","sep21","oct21","nov21","dec21","jan22","feb22","mar22","apr22","may22","jun22","jul22",
                  "aug22","sep22"]

slide_1= slide_1.drop(columns=["jun22"])
slide_1= slide_1.join(pd.Series(slide_1.max(axis=1), name = "max_val", dtype= "float64"))
slide_1 = slide_1.join(maxer(slide_1))
slide_1["growth_1st_last"]= round(((slide_1["sep22"] - slide_1["jun21"])/slide_1["jun21"])+1,1)
slide_1["growth_1st_top"]= round(((slide_1["max_val"] - slide_1["jun21"])/slide_1["jun21"])+1,1)
# slide_1["growth_1st_last%"]= round(((slide_1["may22"] - slide_1["jul21"])/slide_1["jul21"])*100,0)
# slide_1["growth_1st_top%"]= round(((slide_1["max_val"] - slide_1["jul21"])/slide_1["jul21"])*100,0)

slide_1.iloc[:,[0,14,17]]

,jun21,sep22,growth_1st_last
driver_ucounts,257.0,490.0,1.9
trips_count,2079.0,5722.0,2.8
distance_covered,209800.0,538130.2,2.6
clients_ucount,20.0,34.0,1.7
supp_ucount,53.0,67.0,1.3
branch_ucount,59.0,158.0,2.7
gov_ucount,16.0,24.0,1.5
truck_types,17.0,20.0,1.2


##### Slide5__(driver count vs trips count)(by gov) (8 in pres)

In [177]:
geo_1 = df.groupby(["pickup_gov","driver_name","month_year"]).agg({"id":"count"}).unstack(level=2).fillna(0).reset_index() 

geo_1.columns= ["pickup_gov","driver_name","jun21","jul21","aug21","sep21","oct21","nov21","dec21","jan22","feb22","mar22","apr22","may22","jun22","jul22",
                  "aug22","sep22"]

geo_1= geo_1.groupby(["pickup_gov"]).agg({"driver_name":"nunique",
                                   "jun21":"sum",	"jul21":"sum"	,"aug21":"sum",	"sep21":"sum",	"oct21":"sum",
                                   "nov21":"sum",	"dec21":"sum",	"jan22":"sum"	,"feb22":"sum",	"mar22":"sum",
                                   "apr22":"sum",	"may22":"sum","jun22":"sum",
                                         "jul22":"sum", "aug22":"sum","sep22":"sum"}).reset_index()

geo_1["all_trips"] = geo_1.sum(axis=1)
geo_1["growth_1st_last"]= round(((geo_1["may22"] - geo_1["jun21"])/(geo_1["jun21"]+0.01))+1,1)
geo_1.sort_values(by=["all_trips"], ascending = False).reset_index(drop= True)[["pickup_gov","driver_name","all_trips"]]#.to_csv("geo.csv")



,pickup_gov,driver_name,all_trips
0,Al Jizah,1424,25233.0
1,Al Qahirah,875,23329.0
2,Al Qalyubiyah,843,19984.0
3,Ash Sharqiyah,397,5207.0
4,Al Minya,162,2706.0
5,Al Iskandariyah,272,2615.0
6,Asyut,85,2095.0
7,Al Buhayrah,202,1722.0
8,Bani Suwayf,99,1559.0
9,Suhaj,52,1431.0


##### Slide6__(driver count vs trips count)(by gov map) (9 in pres)

In [ ]:
geo_2 = geo_1.sort_values(by=["pickup_gov"], ascending = True).reset_index(drop= True)[["pickup_gov","driver_name","all_trips"]]
geo_2 = geo_2.append(geo_2.loc[10]).reset_index(drop= True).drop(index=10)
geo_2.loc[24.5]= pd.Series({"pickup_gov":"Shamal Sinai","driver_name":0,"all_trips":0}, name=25)
geo_2.sort_index().reset_index(drop= True).to_csv("geo_2.csv")